# Projet de Machine Learning

# Jeu de données

Les données proviennent du concours KAGGLE et font partie du jeu de données intitulé "Gym Members Exercise Dataset", accessible ici : [Gym Members Exercise Dataset](https://www.kaggle.com/datasets/valakhorasani/gym-members-exercise-dataset).

Ce jeu de données offre un aperçu détaillé des habitudes d'exercice, des caractéristiques physiques et des indicateurs de condition physique des membres d’une salle de sport. Il comprend 15 variables observées chez 973 individus fréquentant une salle de sport :

- **Age** : Âge du membre.
- **Gender** : Sexe du membre (variable qualitative avec deux catégories : homme ou femme).
- **Weight_kg** : Poids en kilogrammes.
- **Height_m** : Taille en mètres.
- **Max_BPM** : Fréquence cardiaque maximale (en battements par minute) atteinte pendant les séances d’entraînement.
- **Avg_BPM** : Fréquence cardiaque moyenne durant les séances d’entraînement.
- **Resting_BPM** : Fréquence cardiaque au repos avant l’entraînement.
- **Session_Duration_hours** : Durée de chaque séance d’entraînement en heures.
- **Calories_Burned** : Nombre total de calories brûlées lors de chaque séance.
- **Workout_Type** : Type d’entraînement pratiqué (variable qualitative avec 4 catégories : cardio, musculation, yoga, HIIT).
- **Fat_Percentage** : Pourcentage de masse grasse du membre.
- **Water_Intake_liters** : Quantité d’eau consommée quotidiennement pendant les séances d’entraînement.
- **Workout_Frequency_days_week** : Nombre de séances d’entraînement hebdomadaires (variable qualitative avec 4 catégories : de 2 à 5 séances).
- **Experience_Level** : Niveau d’expérience (variable qualitative avec 3 niveaux : 1 pour débutant, 3 pour expert).
- **BMI** : Indice de masse corporelle (IMC), calculé à partir du poids et de la taille.

L'objectif de ce projet est d'abord de prédire la variable **Calories_Burned** en fonction des autres variables, puis d'estimer la variable **Experience_Level** en utilisant toutes les autres variables disponibles, y compris **Calories_Burned**.


In [161]:
library(ggplot2)
library(ggplot2)
library(gridExtra)
library(dplyr)
library(corrplot)
library(tidyverse)
library(gridExtra)
library(GGally)
library(plotly)
library(reshape2)
library(FactoMineR) 
library(factoextra)
library(glmnet) 
library(ggfortify)
library(pROC)
library(ROCR)
library(ggcorrplot)

## 1. Analyse exploratoire des données
L’objectif dans un premier temps est d’explorer les différentes variables, étape préliminaire indispensable à
l’analyse. Ci-dessous sont précisées quelques questions basiques. Vous pouvez compléter l’analyse selon vos
propres idées.
* Commencez par vérifier la nature des différentes variables et leur encodage. N’oubliez pas de convertir
toutes les variables qualitatives.
* Commencez l’exploration par une analyse descriptive unidimensionnelle des données. Des transformations
des variables quantitatives vous semblent-t-elles pertinentes ?
* Poursuivez avec une analyse descriptive bidimensionnelle. Utilisez des techniques de visualisation: par
exemple les nuages de points (scatterplot), des graphes des correlations, des boîtes à moustaches parallèles,
mosaicplot... Quelles variables semblent liées ?
* Réalisez une analyse en composantes principales des variables explicatives quantitatives et interprétez les
résultats. Visualisez les dépendances éventuelles entre les variables à prédire et les variables explicatives.

### 1.1 Vérifier la nature des différentes variables 

In [80]:
data = read.csv("gym_members_exercise_tracking.csv")
summary(data)

      Age           Gender           Weight..kg.       Height..m.   
 Min.   :18.00   Length:973         Min.   : 40.00   Min.   :1.500  
 1st Qu.:28.00   Class :character   1st Qu.: 58.10   1st Qu.:1.620  
 Median :40.00   Mode  :character   Median : 70.00   Median :1.710  
 Mean   :38.68                      Mean   : 73.85   Mean   :1.723  
 3rd Qu.:49.00                      3rd Qu.: 86.00   3rd Qu.:1.800  
 Max.   :59.00                      Max.   :129.90   Max.   :2.000  
    Max_BPM         Avg_BPM       Resting_BPM    Session_Duration..hours.
 Min.   :160.0   Min.   :120.0   Min.   :50.00   Min.   :0.500           
 1st Qu.:170.0   1st Qu.:131.0   1st Qu.:56.00   1st Qu.:1.040           
 Median :180.0   Median :143.0   Median :62.00   Median :1.260           
 Mean   :179.9   Mean   :143.8   Mean   :62.22   Mean   :1.256           
 3rd Qu.:190.0   3rd Qu.:156.0   3rd Qu.:68.00   3rd Qu.:1.460           
 Max.   :199.0   Max.   :169.0   Max.   :74.00   Max.   :2.000           

Nous observons que:
- **Gender** (Sexe) → *Caractère (qualitative nominale)*
- **Workout_Type** (Type d'entraînement) → *Caractère (qualitative nominale)*
- **Workout_Frequency.days.week.** (Fréquence d'entraînement) → *Qualitative ordinale* (modalités discrètes : 2 à 5 jours)
- **Experience_Level** (Niveau d’expérience) → *Qualitative ordinale* (valeurs : 1 = débutant, 3 = expert)
- **Age** (Âge) → *Quantitative continue*
- **Weight_kg** (Poids) → *Quantitative continue*
- **Height_m** (Taille) → *Quantitative continue*
- **Max_BPM** (Fréquence cardiaque max) → *Quantitative continue*
- **Avg_BPM** (Fréquence cardiaque moyenne) → *Quantitative continue*
- **Resting_BPM** (Fréquence cardiaque au repos) → *Quantitative continue*
- **Session_Duration_hours** (Durée de séance) → *Quantitative continue*
- **Calories_Burned** (Calories brûlées) → *Quantitative continue*
- **Fat_Percentage** (Pourcentage de graisse) → *Quantitative continue*
- **Water_Intake_liters** (Consommation d’eau) → *Quantitative continue*
- **BMI** (Indice de masse corporelle) → *Quantitative continue*

**Cependant, la presentation des variables qualitatives ne sont pas efficases pour les analyser, donc, il faut changer du type des variables qualitatives en facteur**

In [93]:
data[, 'Gender'] <- as.factor(data[, 'Gender'])
data[, 'Workout_Type'] <- as.factor(data[, 'Workout_Type'])
data[, 'Workout_Frequency..days.week.'] <- as.factor(data[, 'Workout_Frequency..days.week.'])
data[, 'Experience_Level'] <- as.factor(data[, 'Experience_Level'])
summary(data)


      Age           Gender     Weight..kg.       Height..m.       Max_BPM     
 Min.   :18.00   Female:462   Min.   : 40.00   Min.   :1.500   Min.   :160.0  
 1st Qu.:28.00   Male  :511   1st Qu.: 58.10   1st Qu.:1.620   1st Qu.:170.0  
 Median :40.00                Median : 70.00   Median :1.710   Median :180.0  
 Mean   :38.68                Mean   : 73.85   Mean   :1.723   Mean   :179.9  
 3rd Qu.:49.00                3rd Qu.: 86.00   3rd Qu.:1.800   3rd Qu.:190.0  
 Max.   :59.00                Max.   :129.90   Max.   :2.000   Max.   :199.0  
    Avg_BPM       Resting_BPM    Session_Duration..hours. Calories_Burned 
 Min.   :120.0   Min.   :50.00   Min.   :0.500            Min.   : 303.0  
 1st Qu.:131.0   1st Qu.:56.00   1st Qu.:1.040            1st Qu.: 720.0  
 Median :143.0   Median :62.00   Median :1.260            Median : 893.0  
 Mean   :143.8   Mean   :62.22   Mean   :1.256            Mean   : 905.4  
 3rd Qu.:156.0   3rd Qu.:68.00   3rd Qu.:1.460            3rd Qu.:1076.0

### 1.2 Analyse descriptive unidimensionnelle

In [113]:
cat_cols = c('Gender','Workout_Type', 'Workout_Frequency..days.week.', 'Experience_Level')
cat = data[cat_cols]
num <- data %>% select(-all_of(cat_cols))

In [173]:
ggcorrplot(cor(num), 
           method = "square",
           type = "lower",
           lab = TRUE,
           colors = c("blue", "white", "red"),
           outline.color = "black",
           tl.cex = 12)

In [151]:
ggpairs(num)